In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from os.path import join as oj
import sys
sys.path.append('../src')
import numpy as np
import pandas as pd
import data, data_psrc
from tqdm import tqdm
from copy import deepcopy
from pecarn_predict import pecarn_rule_predict

# load data
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = \
    data.load_it_all(dummy=False, impute=True, frac_missing_allowed=1, use_processed=False)
# d = df_pecarn # [common_feats]
d = df_psrc

# get data with FAST
# df = data_psrc.get_data(use_processed=False, dummy=True) #use_processed_feats=True,)
# df['FAST (choice=not performed)'] = df['FAST (choice=0t performed)']
# fast_study_cohort, _ = data_psrc.get_FAST(df) 
# d = d[fast_study_cohort]


d['Age<2'] = d['Age'] < 2

In [14]:
d.columns

Index(['Age', 'Hispanic ethnicity',
       'Mechanism of injury (choice=Assault/struck)',
       'Mechanism of injury (choice=ATV injury)',
       'Mechanism of injury (choice=Bike crash)',
       'Mechanism of injury (choice=Bike struck by auto)',
       'Mechanism of injury (choice=Fall > 10 ft. height)',
       'Mechanism of injury (choice=Golf cart injury)',
       'Mechanism of injury (choice=Motorcycle/dirt bike crash)',
       'Mechanism of injury (choice=MVC)',
       ...
       'Hypotension_no', 'Hypotension_yes', 'GCSScore_Full_no',
       'GCSScore_Full_yes', 'Age<2_no', 'Age<2_yes', 'CostalTender_no',
       'CostalTender_yes', 'dset', 'Age<2'],
      dtype='object', length=424)

# check bivariable associations

In [10]:
# can also check vars that were not used
rules = [
    ('AbdTrauma_or_SeatBeltSign', ['yes']),
    ('GCSScore', range(14)),
#     ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
    ('AbdTenderDegree', ['None']),    
    ('AbdTenderDegree', ['Mild']),    
    ('AbdTenderDegree', ['Moderate']),    
    ('AbdTenderDegree', ['Severe']),        
    ('ThoracicTrauma', ['yes']),        
    ('AbdomenPain', ['yes']),
    ('DecrBreathSound', ['yes']),
    ('VomitWretch', ['yes']),
    
    # not used in rule
    ('Age<2', [True]),    
    ('AbdDistention', ['yes']),
    ('MOI', ['Motor vehicle collision']),
    ('DistractingPain', ['yes']),    
    ('CostalTender', ['yes']),
    ('ThoracicTender', ['yes']),    
    ('Hypotension', ['yes']),        
    ('AbdomenTender', [1]),    
]
index = ['NA', 'IAIw/I', 'No IAIw/I', 'Difference']
o = data.outcome_def
r = {k + ' = ' + str(v[0]) : [] for k, v in rules}
io = d[o] == 1
print('num IwI and key / num IwI (not nan) \t num without IwI and key / num without IwI (not nane) ')
for rule in rules:
    k, vals = rule
    kk = k + ' = ' + str(vals[0])
    ik = d[k].isin(vals) # d[o]==1
    inan = d[k].isna() | (d[k] == 'unknown')
    r[kk].append(np.sum(inan))   
    num1 = np.sum(d[ik][o]==1)
    denom1 = (io & ~inan).sum()
    r[kk].append(f'{num1}/{denom1}')
    num2 = np.sum(d[ik][o]==0)
    denom2 = (~io & ~inan).sum()
    r[kk].append(f'{num2}/{denom2}')
    r[kk].append(f'{num1/denom1 - num2/denom2:0.3f}')    
r = pd.DataFrame.from_dict(r)
r.index = index
r.transpose()

num IwI and key / num IwI (not nan) 	 num without IwI and key / num without IwI (not nane) 


KeyError: 'AbdTrauma_or_SeatBeltSign'

# recreate / evaluate rule

**look at pecarn patients**

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_pecarn)
assert missed_patients.shape[0] == 6, 'should find 6 low-risk patients'    
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')


# save out missed patients
df_pecarn.loc[missed_patients.index].to_csv('../reports/missed_patients_pecarn.csv')
print('\nmissed patients:')
missed_patients[['Age', 'Sex', 'MOI']]

Initial                   203 / 12044
AbdTrauma_or_SeatBeltSign  91 / 10081	112 / 1963 (5.7)
GCSScore                   53 /  9255	 38 /  826 (4.6)
AbdTenderDegree            17 /  6767	 36 / 2488 (1.4)
ThoracicTrauma             11 /  5805	  6 /  962 (0.6)
AbdomenPain                 9 /  5469	  2 /  336 (0.6)
DecrBreathSound             8 /  5435	  1 /   34 (2.9)
VomitWretch                 6 /  5040	  2 /  395 (0.5)
sensitivity 97.04
specificity 42.51

missed patients:


,Age,Sex,MOI
203,2,M,Fall down stairs
4349,2,M,Pedestrian/bicyclist struck by moving vehicle
5977,16,M,Motorcycle/ATV/Scooter collision
9419,17,F,Motor vehicle collision
9436,17,M,Motor vehicle collision
9971,17,M,Motor vehicle collision


**look at psrc patients**

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_psrc)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')

# save out missed patients
df_psrc.loc[missed_patients.index].to_csv('../reports/missed_patients_psrc.csv')
missed_patients[['Age', 'MOI']]

Initial                   62 / 2188
AbdTrauma_or_SeatBeltSign  18 /  1708	 44 /  480 (9.2)
GCSScore                    8 /  1441	 10 /  267 (3.7)
AbdTenderDegree             2 /  1216	  6 /  225 (2.7)
ThoracicTrauma              2 /  1123	  0 /   93 (0.0)
AbdomenPain                 2 /   993	  0 /  130 (0.0)
DecrBreathSound             2 /   993	  0 /    0 (nan)
VomitWretch                 2 /   937	  0 /   56 (0.0)
sensitivity 96.77
specificity 43.98


/accounts/projects/vision/chandan/iai-interpretability/notebooks/pecarn_predict.py:38: RuntimeWarning: invalid value encountered in long_scalars
  prob = num2 / denom2


,Age,MOI
1164,1.0,Motor vehicle collision
1755,6.0,Fall from an elevation


# look at errors for subgroups
**here we consider the riskier subgroups (young age, female)**

In [ ]:
# filter patients by age or sex
# d = df_pecarn[df_pecarn['Sex'] == 'F']
d = df_pecarn[df_pecarn['Age'] <= 15]
# d = df_pecarn[df_pecarn['Age'] <= 2]


low_risk_patients, missed_patients, stats = pecarn_rule_predict(d)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')

print('\nmissed patients:')
missed_patients[['Age', 'Sex', 'MOI']]

Initial                   166 / 9907
AbdTrauma_or_SeatBeltSign  74 /  8226	 92 / 1681 (5.5)
GCSScore                   42 /  7501	 32 /  725 (4.4)
AbdTenderDegree            12 /  5510	 30 / 1991 (1.5)
ThoracicTrauma              7 /  4746	  5 /  764 (0.7)
AbdomenPain                 5 /  4460	  2 /  286 (0.7)
DecrBreathSound             4 /  4438	  1 /   22 (4.5)
VomitWretch                 2 /  4094	  2 /  344 (0.6)
sensitivity 98.80
specificity 42.01

missed patients:


,Age,Sex,MOI
203,2,M,Fall down stairs
4349,2,M,Pedestrian/bicyclist struck by moving vehicle


# look at errors on splits

In [ ]:
train_idxs = d.cv_fold.isin(data.pecarn_train_idxs)
test_idxs1 = d.cv_fold.isin(data.pecarn_test_idxs)

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(d[train_idxs])
stats

Initial                   133 / 7985
AbdTrauma_or_SeatBeltSign  62 /  6747	 71 / 1238 (5.7)
GCSScore                   37 /  6182	 25 /  565 (4.4)
AbdTenderDegree            11 /  4538	 26 / 1644 (1.6)
ThoracicTrauma              6 /  3895	  5 /  643 (0.8)
AbdomenPain                 4 /  3664	  2 /  231 (0.9)
DecrBreathSound             3 /  3638	  1 /   26 (3.8)
VomitWretch                 2 /  3401	  1 /  237 (0.4)


{'fn': 2,
 'tp': 131,
 'tn': 3399,
 'fp': 4453,
 'sensitivity': 98.49624060150376,
 'specificity': 43.288334182373916,
 'ppv': 0.028577661431064574,
 'npv': 0.9994119376653925,
 'lr+': 1.73678976241412,
 'lr-': 0.03473821358338476,
 'acc': 0.44207889793362554,
 'roc_auc': 0.8197882633225959,
 'f1': 0.0555437778248887,
 'brier_score': 0.015943728191817625}

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(d[test_idxs1])
stats

Initial                   70 / 4059
AbdTrauma_or_SeatBeltSign  29 /  3334	 41 /  725 (5.7)
GCSScore                   16 /  3073	 13 /  261 (5.0)
AbdTenderDegree             6 /  2229	 10 /  844 (1.2)
ThoracicTrauma              5 /  1910	  1 /  319 (0.3)
AbdomenPain                 5 /  1805	  0 /  105 (0.0)
DecrBreathSound             5 /  1797	  0 /    8 (0.0)
VomitWretch                 4 /  1639	  1 /  158 (0.6)


{'fn': 4,
 'tp': 66,
 'tn': 1635,
 'fp': 2354,
 'sensitivity': 94.28571428571428,
 'specificity': 40.98771621960391,
 'ppv': 0.02727272727272727,
 'npv': 0.9975594874923734,
 'lr+': 1.5977303070761013,
 'lr-': 0.13941459152468333,
 'acc': 0.4190687361419069,
 'roc_auc': 0.8059789420907496,
 'f1': 0.05301204819277108,
 'brier_score': 0.016480946134339907}

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_psrc)
stats

Initial                   62 / 2188
AbdTrauma_or_SeatBeltSign  18 /  1708	 44 /  480 (9.2)
GCSScore                    8 /  1441	 10 /  267 (3.7)
AbdTenderDegree             2 /  1216	  6 /  225 (2.7)
ThoracicTrauma              2 /  1123	  0 /   93 (0.0)
AbdomenPain                 2 /   993	  0 /  130 (0.0)
DecrBreathSound             2 /   993	  0 /    0 (nan)
VomitWretch                 2 /   937	  0 /   56 (0.0)


/accounts/projects/vision/chandan/iai-interpretability/notebooks/pecarn_predict.py:39: RuntimeWarning: invalid value encountered in long_scalars
  #         print('prob', num2, denom2, prob, 'end')


{'fn': 2,
 'tp': 60,
 'tn': 935,
 'fp': 1191,
 'sensitivity': 96.7741935483871,
 'specificity': 43.97930385700847,
 'ppv': 0.047961630695443645,
 'npv': 0.9978655282817502,
 'lr+': 1.72747217030958,
 'lr-': 0.07334828359496284,
 'acc': 0.4547531992687386,
 'roc_auc': 0.8269429187023943,
 'f1': 0.0913937547600914,
 'brier_score': 0.02624673943693975}